In [ ]:
import cherrypy
import json
import redis

REDIS_HOST = ''
REDIS_PORT = 
REDIS_USERNAME = ''
REDIS_PASSWORD = ''

redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)

class Status(object):
    exposed = True
    def GET(self, *path, **query):
        return json.dumps({'status': 'online'})

class Sensors(object):
    exposed = True
    def POST(self, *path, **query):
        body = cherrypy.request.body.read()
        body_dict = json.loads(body.decode())
        mac_address = body_dict.get('mac_address')
        if not mac_address:
            raise cherrypy.HTTPError(400, 'Missing MAC address')
        try:
            redis_client.ts().create(f'{mac_address}:temperature')
            redis_client.ts().create(f'{mac_address}:humidity')
        except redis.ResponseError:
            pass # Already exists
        return

#Historical Data Resource
class HistoricalData(object):
    exposed = True

    def GET(self, mac_address, **query): 
        # Validate Parameters
        if not mac_address:
            raise cherrypy.HTTPError(400, "Missing MAC address.") 
        
        count = query.get('count')
        if not count:
            raise cherrypy.HTTPError(400, "Missing count parameter.") 
        
        try:
            count = int(count)
            if count <= 0:
                raise ValueError
        except ValueError:
            raise cherrypy.HTTPError(400, "Count must be a positive integer.") 

        # Check if sensor exists in DB
        if not redis_client.exists(f'{mac_address}:temperature'):
             raise cherrypy.HTTPError(404, "MAC address not found.") 

        # Retrieve Data using revrange 
        # Returns list of tuples [(timestamp, value), ...]
        temp_data = redis_client.ts().revrange(f'{mac_address}:temperature', '-', '+', count=count)
        hum_data = redis_client.ts().revrange(f'{mac_address}:humidity', '-', '+', count=count)

        # Format Response
        timestamps = [int(t) for t, v in temp_data]
        temperatures = [int(v) for t, v in temp_data]
        humidities = [int(v) for t, v in hum_data]
        
        # Sort back to chronological order if revrange reversed them (optional but good for plots)
        timestamps.reverse()
        temperatures.reverse()
        humidities.reverse()

        response = {
            "mac_address": mac_address,
            "timestamp": timestamps,
            "temperature": temperatures,
            "humidity": humidities
        }
        
        return json.dumps(response)

# Main server
if __name__ == '__main__':
    conf = {'/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}}
    cherrypy.tree.mount(Status(), '/status', conf)
    cherrypy.tree.mount(Sensors(), '/sensors', conf)
    # Mount the new endpoint
    cherrypy.tree.mount(HistoricalData(), '/data', conf) 
    
    cherrypy.config.update({'server.socket_host': '0.0.0.0'})
    cherrypy.config.update({'server.socket_port': 8080})
    cherrypy.engine.start()
    cherrypy.engine.block()

[19/Dec/2025:14:21:42] ENGINE Bus STARTING
[19/Dec/2025:14:21:42] ENGINE Started monitor thread 'Autoreloader'.
[19/Dec/2025:14:21:42] ENGINE Serving on http://0.0.0.0:8080
[19/Dec/2025:14:21:42] ENGINE Bus STARTED
10.236.32.153 - - [19/Dec/2025:14:22:36] "GET /status HTTP/1.1" 200 20 "" "python-requests/2.32.5"
10.236.51.220 - - [19/Dec/2025:14:22:37] "POST /sensors HTTP/1.1" 200 - "" "python-requests/2.32.5"
10.236.48.105 - - [19/Dec/2025:14:22:37] "GET /data/26:99:64:91:45:14?count=10 HTTP/1.1" 200 312 "" "python-requests/2.32.5"
10.236.32.153 - - [19/Dec/2025:14:22:56] "GET /status HTTP/1.1" 200 20 "" "python-requests/2.32.5"
10.236.51.220 - - [19/Dec/2025:14:22:56] "POST /sensors HTTP/1.1" 200 - "" "python-requests/2.32.5"
10.236.45.65 - - [19/Dec/2025:14:22:56] "GET /data/26:99:64:91:45:14?count=10 HTTP/1.1" 200 312 "" "python-requests/2.32.5"
[19/Dec/2025:14:25:24] ENGINE Keyboard Interrupt: shutting down bus
[19/Dec/2025:14:25:24] ENGINE Bus STOPPING
[19/Dec/2025:14:25:24] ENGI

KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=37068574-a210-48cc-b0b2-9175710228d6' target="_blank">

Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>